# 分块

我们用于实体识别的基本技术是**分块**，即分割和标注多个单词组成的序列。如下图所示，小框显示词级的标识符和词性标注，而大框则显示较高级别的分块，块与块之间没有重叠。

![chunk-segmentation.png](resources/chunk-segmentation.png)

## 名词短语分块
名词短语分块，又称为 NP-分块，即寻找句子中单独的名词短语对应的块。这里我们定义一个 NP 块不能包含其它的 NP 块，修饰一个名词的任何介词短语或从句都不包括在 NP 块内，因此 NP 块一般要比完整的名词短语小。

例如：the market for system-management software for Digital's hardware 是一个完整的名词短语，它中间有一个简单的 NP 块 the market。

为了对句子进行分块，我们首先需要定义一个**块语法**，这里我们用正则表达式规则定义一个简单的语法：一个 NP 块由一个可选的限定词（DT）后面跟着任意数目的形容词（JJ）然后是一个名词（NN）组成的。使用此语法，我们创建了一个块分析器：

In [1]:
import nltk

sentence = [('the', 'DT'), ('little', 'JJ'), ('yellow', 'JJ'), ('dog', 'NN'),
            ('barked', 'VBD'), ('at', 'IN'), ('the', 'DT'), ('cat', 'NN')]
grammar = 'NP: {<DT>?<JJ>*<NN>}'
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


In [2]:
result.draw()

我们可以通过 draw 方法来输出分块结果树：

![ch07-tree-1.png](resources/ch07-tree-1.png)

## 标记模式
块语法中的规则使用**标记模式**来描述已标注的词的序列，一个标记模式是一个用尖括号分割的词性标记序列，如 &lt;DT&gt;?&lt;JJ&gt;*&lt;NN&gt;，类似于正则表达式模式。

我们对这个标记模式进行轻微的改进，如 &lt;DT&gt;?&lt;JJ.*&gt;*&lt;NN.*&gt;+，就可以识别任何一个已可选的限定词开头，后面跟零个或多个任何类型的形容词（包括相对形容词，如 earlier/JJR ），后面跟一个或多个任何类型的名词的标识符序列。

## 用正则表达式分块
[RegexpParser](https://www.nltk.org/_modules/nltk/chunk/regexp.html#RegexpParser) 正则表达式分块器是一个基于语法规则的分块器，它可以接受一组块语法规则并轮流加以应用，依次更新块结构。

下例显示了一个由 2 个规则组成的块语法：第一条规则匹配一个可选的限定词或所有格代词，零个或多个形容词，然后跟一个名词；第二条规则匹配一个或多个专有名词。它们都定义了一个 NP 块。

In [3]:
grammar = r"""
    NP: 
        {<DT|PP\$>?<JJ>*<NN>}
        {<NNP>+}
"""
cp = nltk.RegexpParser(grammar)
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"),
            ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]
print(cp.parse(sentence))

(S
  (NP Rapunzel/NNP)
  let/VBD
  down/RP
  (NP her/PP$ long/JJ golden/JJ hair/NN))


如果标记模式匹配位置重叠，最左边的匹配优先：

In [4]:
nouns = [("money", "NN"), ("market", "NN"), ("fund", "NN")]
grammar = 'NP: {<NN><NN>} # Chunk two consecutive nouns'
cp = nltk.RegexpParser(grammar)
print(cp.parse(nouns))

(S (NP money/NN market/NN) fund/NN)


## 探索文本语料库
在 5.2 节中，我们采用了比较繁琐的方式来从已标注的语料库中提取匹配的特定的词性标记序列的短语，这里我们可以使用分块器更容易地做同样的工作：

In [5]:
cp = nltk.RegexpParser('CHUNK: {<V.*><TO><V.*>}')
brown = nltk.corpus.brown
chunks = []
for sent in brown.tagged_sents():
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK':
            chunks.append(subtree)
for subtree in chunks[:5]:
    print(subtree)

(CHUNK combined/VBN to/TO achieve/VB)
(CHUNK continue/VB to/TO place/VB)
(CHUNK serve/VB to/TO protect/VB)
(CHUNK wanted/VBD to/TO wait/VB)
(CHUNK allowed/VBN to/TO place/VB)


## 加缝隙
RegexpParser 支持加缝隙操作，即从一大块中去除一个标识符序列的过程。缝隙 Chink 的定义为不包含在一个块中的标识符序列，通过开头的 } 和结束的 { 来标识。

在下面的例子中，首先通过 &lt;.*&gt;+ 将所有的标识符收集在一个 NP 块中，然后通过缝隙操作去掉 VBD 或 IN 组成的序列，分割成两个 NP 块，注意语法规则是按顺序依次对整条语句进行的：

In [6]:
grammar = r"""
    NP:
        {<.*>+}           # Chunk everything
        }<VBD|IN>+{       # Chink sequences of VBD and IN
"""
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),("dog", "NN"), 
            ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)
print(cp.parse(sentence))

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


## 块的表示：标记与树
块的表示有两种，一种是树结构 [nltk.tree.Tree](https://www.nltk.org/_modules/nltk/tree.html#Tree)，这是 RegexpParser.parse() 方法返回的默认形式；另一种广泛的表示方式是 **IOB 标记**。在这种方式中，每个标识符用三个特殊的块标签之一标注，I（inside，内部），O（outside，外部）或 B（begin，开始）。一个标识符被标注为 B，如果它标志着一个块的开始，块内的标识符子序列被标注为 I，所有其他标识符被标注为 O。B 和 I 标记会加上块类型的后缀，如 B-NP，I-NP 等。

![chunk-tagrep.png](resources/chunk-tagrep.png)

In [7]:
print('Tree =>')
print(result)

from nltk import tree2conlltags
iob_tags = tree2conlltags(result)
print('\nIOB Tags =>')
print(iob_tags)

Tree =>
(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))

IOB Tags =>
[('the', 'DT', 'B-NP'), ('little', 'JJ', 'I-NP'), ('yellow', 'JJ', 'I-NP'), ('dog', 'NN', 'I-NP'), ('barked', 'VBD', 'O'), ('at', 'IN', 'O'), ('the', 'DT', 'B-NP'), ('cat', 'NN', 'I-NP')]
